# Preparing Data

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [4]:
players = pd.read_csv('players.csv')
valuations = pd.read_csv('player_valuations.csv')
appearances = pd.read_csv('appearances.csv')
clubs = pd.read_csv('clubs.csv')

In [5]:
Player_Season_Stats = appearances.groupby('player_id').sum()
Player_Season_Stats['Total_Contributions'] = Player_Season_Stats['goals'] + Player_Season_Stats['assists']
Player_Season_Stats['Contributions_per_90'] = (90 * Player_Season_Stats[
    'Total_Contributions']) / Player_Season_Stats['minutes_played']

In [6]:
current_value = valuations.groupby('player_id').last()
expanded_value = pd.merge(current_value, Player_Season_Stats, on= 'player_id')

In [7]:
players_stats_w_finance = expanded_value.drop(['datetime', 'dateweek','game_id','yellow_cards','red_cards'], axis = 1)
players_stats_w_finance['league_id'] = players_stats_w_finance['player_club_domestic_competition_id']
df_players = players_stats_w_finance.drop(['player_club_domestic_competition_id'], axis = 1)

In [8]:
Players_ID_Index = players.groupby(['player_id']).last()
DOB = Players_ID_Index.date_of_birth
df_players['position'] = Players_ID_Index['position']

In [9]:
def arange(word):
    new_word = word[5:7]+'/'+word[8:]+'/'+word[:4]
    return new_word

Players_ID_Index['DOB'] =  Players_ID_Index.iloc[:,6].apply(str).apply(arange)

In [10]:
Players_ID_Index['DOB'] = pd.to_datetime(Players_ID_Index['DOB'], errors='coerce')

In [11]:
Players_ID_Index['today']= pd.Timestamp.today().strftime('%Y-%m-%d')

In [12]:
df_players.columns

Index(['date', 'market_value', 'current_club_id', 'player_club_id', 'goals',
       'assists', 'minutes_played', 'Total_Contributions',
       'Contributions_per_90', 'league_id', 'position'],
      dtype='object')

In [13]:
additions = ['goals','assists', 'minutes_played', 'Total_Contributions','league_id']

In [14]:
Players_ID_Index[additions] = df_players[additions]

In [15]:
# This function converts given date to age
def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))
  
Players_ID_Index['AGE'] = Players_ID_Index['DOB'].apply(age)

In [31]:
df_small = Players_ID_Index[['last_season', 'current_club_id', 'name','country_of_citizenship', 'position', 
                             'market_value_in_gbp', 'AGE', 'height_in_cm','goals','assists', 
                             'minutes_played', 'Total_Contributions','league_id','foot', 'sub_position']]

In [17]:
df = df_small.dropna()

# Building Model 

In [86]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [87]:
df2 = df.loc[df['position'] == 'Attack']
df3 = df_small.loc[df_small['position'] == 'Attack']
df3['foot'].replace({None: "Both"}, inplace=True)
df3 = df3.dropna()
df3

/Users/maxrossi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,last_season,current_club_id,name,country_of_citizenship,position,market_value_in_gbp,AGE,height_in_cm,goals,assists,minutes_played,Total_Contributions,league_id,foot,sub_position
player_id,,,,,,,,,,,,,,,
215,2015,1084,roque-santa-cruz,Paraguay,Attack,225000.0,41.0,193,8.0,1.0,1584.0,9.0,ES1,Right,Centre-Forward
2453,2014,127,mahir-saglik,Turkey,Attack,45000.0,39.0,178,1.0,1.0,313.0,2.0,L1,Left,Centre-Forward
3455,2022,5,zlatan-ibrahimovic,Sweden,Attack,2700000.0,41.0,195,130.0,44.0,14712.0,174.0,IT1,Both,Centre-Forward
4188,2021,2420,ricardo-quaresma,Portugal,Attack,270000.0,39.0,175,38.0,62.0,17075.0,100.0,PO1,Right,Right Winger
4276,2014,506,carlos-tevez,Argentina,Attack,270000.0,38.0,171,29.0,10.0,4039.0,39.0,IT1,Right,Second Striker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980576,2022,1420,amine-salama,France,Attack,360000.0,22.0,192,1.0,1.0,185.0,2.0,FR1,Right,Centre-Forward
987700,2022,16704,jonathan-okoronkwo,Nigeria,Attack,270000.0,19.0,175,0.0,0.0,133.0,0.0,RU1,Right,Centre-Forward
988710,2022,3205,baran-gezek,Turkey,Attack,45000.0,17.0,0,0.0,0.0,16.0,0.0,TR1,Both,Attacking Midfield


In [88]:
# target 
y = df3.market_value_in_gbp

# dependent variables
features = ['country_of_citizenship', 'league_id',
            'height_in_cm', 'AGE', 'goals','assists', 'minutes_played', 'Total_Contributions', 'foot', 'sub_position']  
    
    #plan to use above but need create ID's for countries... and positions...
    
    
#features = ['current_club_id', 'AGE', 'height_in_cm', 
           # 'goals','assists', 'minutes_played', 'Total_Contributions']   
    
x = df3[features]

In [89]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(x, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [91]:
# defining model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0)

In [92]:
#Creating and Evaluating Pipeline

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['height_in_cm', 'AGE',
                                                   'goals', 'assists',
                                                   'minutes_played',
                                                   'Total_Contributions']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['foot', 'sub_position'])])),
                ('model

In [93]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, x, y,
                              cv=6,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [1512395.91075055 1573234.81582781 1950846.21459161 2429975.83917219
 2562435.75646304 2458091.49118232]


In [94]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
2081163.33799792


# Analysing the outcomes 

In [109]:
# adding back prices to dataframe
numbers = df3['market_value_in_gbp']
df4 = X_train.join(numbers)

,foot,sub_position,height_in_cm,AGE,goals,assists,minutes_played,Total_Contributions,market_value_in_gbp
player_id,,,,,,,,,
236510,Left,Left Winger,179,25.0,6.0,11.0,2936.0,17.0,1800000.0
193158,Both,Left Winger,170,29.0,15.0,15.0,6327.0,30.0,135000.0
406891,Right,Left Winger,184,24.0,2.0,0.0,578.0,2.0,9000.0
791528,Left,Attacking Midfield,180,21.0,0.0,0.0,486.0,0.0,270000.0
666563,Right,Centre-Forward,175,19.0,0.0,0.0,9.0,0.0,180000.0
...,...,...,...,...,...,...,...,...,...
594991,Right,Centre-Forward,188,24.0,14.0,5.0,2619.0,19.0,10800000.0
324329,Right,Right Winger,171,27.0,6.0,3.0,2618.0,9.0,315000.0
165793,Right,Left Winger,174,27.0,15.0,6.0,3261.0,21.0,10800000.0


In [ ]:
#for further examination- (Comparison DataFrame)
preds = my_pipeline.predict(X_train)

In [110]:
Value_comparison = pd.DataFrame({'Player_Id': df4.index, 'market_value_in_gbp' : df4['market_value_in_gbp'],
                       'Predicted_Price': preds})
Value_comparison['Underestimated Value'] = (Value_comparison['Predicted_Price'] - 
                                            Value_comparison['market_value_in_gbp'])

In [119]:
# Most undervalued players??
Value_comparison.sort_values(by=['Underestimated Value'], ascending = False).head()

,Player_Id,market_value_in_gbp,Predicted_Price,Underestimated Value
player_id,,,,
324503,324503,5400000.0,19080000.0,13680000.0
125781,125781,31500000.0,43641000.0,12141000.0
412932,412932,12600000.0,22783500.0,10183500.0
434675,434675,27000000.0,36936000.0,9936000.0
58864,58864,13500000.0,22401000.0,8901000.0


In [149]:
Player_names = players.set_index('player_id')

In [155]:
# player ID to find player 
x = 324503
Player_names.loc[x]

last_season                                                                 2022
current_club_id                                                             1090
name                                                           vangelis-pavlidis
pretty_name                                                    Vangelis Pavlidis
country_of_birth                                                          Greece
country_of_citizenship                                                    Greece
date_of_birth                                                         1998-11-21
position                                                                  Attack
sub_position                                                      Centre-Forward
foot                                                                        Both
height_in_cm                                                                 186
market_value_in_gbp                                                    5400000.0
highest_market_value_in_gbp 

In [ ]:
# 324503 = Vangelis Pavlidis - I don't know who this is but intresting 
# 125781 = Antoine Greizman - moved to athletico on loan 
# 412932 = Myron Boadu - man united were intrested in him
# 434675 = Cody Gakpo - Heavily transfer linked all summer
# 58864 = Aubameyang - move to chelsea from barcelona for about £13 million 